In [1]:
import invisible_cities.reset.reset_utils as reset

from invisible_cities.core.system_of_units import pes, mm, mus, ns
import invisible_cities.reco.corrections    as corrf
import invisible_cities.database.load_db as dbf 
import reset_functions_event as rstf

# Initialize RESET
iterations = 100
pitch      = 10
run_number = 4495
nsipms     = 1792
npmts      = 1 
dist       = 20. 
sipm_dist  = 20. 
pmt_dist   = 200 # max distance included in the param file
sipm_thr   = 5.
#x_size     = 2.
#y_size     = 2.

x_size     = 10.
y_size     = 10.

rmax       = 198 #
slice_width = 2.
pmt_param  = "/home/jmbenlloch/reset_data/mapas/PMT_Map_corr_keV.h5"
sipm_param = "/home/jmbenlloch/reset_data/mapas/SiPM_Map_corr_z0.0_keV.h5"

pmap_file = '4495_21215.h5'

pmap_conf = {}
pmap_conf['s1_emin'] = 54 * pes 
pmap_conf['s1_emax'] = 2400. * pes 
pmap_conf['s1_wmin'] = 7*25 * ns
pmap_conf['s1_wmax'] = 16*25 * ns
pmap_conf['s1_hmin'] = 12. * pes 
pmap_conf['s1_hmax'] = 400. * pes 
pmap_conf['s1_num'] = 1 
pmap_conf['s1_ethr'] = 0.5 * pes 

pmap_conf['s2_emin'] = 200000. * pes 
pmap_conf['s2_emax'] = 2.e6 * pes 
pmap_conf['s2_wmin'] = 4.* mus 
pmap_conf['s2_wmax'] = 500. * mus 
pmap_conf['s2_hmin'] = 600. * pes 
pmap_conf['s2_hmax'] = 60000. * pes 
pmap_conf['s2_numMin'] = 1 
pmap_conf['s2_numMax'] = 3 
pmap_conf['s2_ethr'] = 0. * pes 
pmap_conf['s2_nsipmmin'] = 1 
pmap_conf['s2_nsipmmax'] = 1792

# Read file and select peaks
selector = reset.refresh_selector(pmap_conf)
s1s, s2s, s2sis, peaks = reset.load_and_select_peaks(pmap_file, 21215, selector)

# Lifetime correction
ZCorr = corrf.LifetimeCorrection(1093.77, 23.99)

# Sensors info
data_sipm = dbf.DataSiPM(run_number)

# Take the first peak
peak = next(iter(peaks))
evt = 21215
sipm_thr = sipm_thr

# Prepare data
voxels_data, slices, energies, zs = reset.prepare_data(s1s, s2s, s2sis, slice_width, evt, peak, data_sipm, nsipms, sipm_thr, dist, ZCorr)
slices_start = reset.slices_start(voxels_data, x_size, y_size)


#rst = rstf.RESET(run_number, nsipms, npmts, dist, sipm_dist, pmt_dist, sipm_thr, x_size, y_size, rmax, sipm_param, pmt_param)
#voxels, slices = rst.run(voxels, slices, energies, slices_start, iterations)
#reset.write_hdf5(voxels, slices, zs) 
#rst._destroy_context()

In [2]:
import invisible_cities.reset.reset_serial as rst_serial

# Create voxels

In [3]:
rst_voxels = rst_serial.create_voxels(voxels_data, slices_start, x_size, y_size, rmax)

In [4]:
rst_voxels

ResetVoxels(nvoxels=13826, voxels=array([(115., 25., 6.3964443), (115., 35., 6.3964443),
       (115., 45., 6.3964443), ..., (105., 55., 4.248336 ),
       (105., 65., 4.248336 ), (105., 75., 4.248336 )],
      dtype=[('x', '<f4'), ('y', '<f4'), ('E', '<f4')]), slice_ids=array([ 0,  0,  0, ..., 65, 65, 65], dtype=int32), slice_start=array([    0,    67,   173,   470,   872,  1404,  1717,  2202,  3113,
        4139,  5135,  6239,  7132,  7903,  8412,  8709,  8957,  9191,
        9343,  9479,  9600,  9641,  9680,  9719,  9768,  9834,  9913,
       10016, 10069, 10115, 10156, 10223, 10303, 11096, 11171, 11255,
       11330, 11405, 11468, 11537, 11657, 11720, 11805, 11868, 11937,
       12012, 12088, 12175, 12259, 12333, 12514, 12601, 12678, 12755,
       12827, 12898, 12953, 13012, 13059, 13095, 13123, 13170, 13204,
       13244, 13298, 13584, 13826], dtype=int32), address=array([    0,     1,     2, ..., 13824, 13825, 13826]))

# CUDA version

In [5]:
rst = rstf.RESET(run_number, nsipms, npmts, dist, sipm_dist, pmt_dist, sipm_thr, x_size, y_size, rmax, sipm_param, pmt_param)

SensorsParams(xmin=-195.0, ymin=-195.0, step=10.0, nbins=40, params=array([(-195., -195., 0.), (-195., -185., 0.), (-195., -175., 0.), ...,
       ( 195.,  175., 0.), ( 195.,  185., 0.), ( 195.,  195., 0.)],
      dtype=[('x', '<f4'), ('y', '<f4'), ('factor', '<f4')]))


In [6]:
import pycuda.driver as cuda

nslices = int(voxels_data.xmin.shape[0])
voxels_data_d = rstf.copy_voxels_data_h2d(voxels_data)
slices_data_d = rstf.copy_slice_data_h2d(slices)
 
slices_start_nc_d = cuda.to_device(slices_start)

 
rst_voxels_d, slice_ids_h = rstf.create_voxels(rst.cudaf, voxels_data_d, rst.xsize,
        rst.ysize, rst.rmax, rst.max_voxels, nslices,
        slices_start_nc_d, int(slices_start[-1]), slices_start)


In [7]:
rst_voxels_d

ResetVoxels(nvoxels=13826, voxels=<pycuda._driver.DeviceAllocation object at 0x7fb5b61eb9e0>, slice_ids=<pycuda._driver.DeviceAllocation object at 0x7fb5b61ebb70>, slice_start=<pycuda._driver.DeviceAllocation object at 0x7fb5b61eb8f0>, address=array([    0,     1,     2, ..., 13824, 13825, 13826], dtype=int32))

In [8]:
rst_voxels_h = rstf.copy_reset_voxels_d2h(rst_voxels_d, nslices)

In [9]:
rst_voxels_h

ResetVoxels(nvoxels=13826, voxels=array([(115., 25., 6.3964443), (115., 35., 6.3964443),
       (115., 45., 6.3964443), ..., (105., 55., 4.248336 ),
       (105., 65., 4.248336 ), (105., 75., 4.248336 )],
      dtype=[('x', '<f4'), ('y', '<f4'), ('E', '<f4')]), slice_ids=array([ 0,  0,  0, ..., 65, 65, 65], dtype=int32), slice_start=array([    0,    67,   173,   470,   872,  1404,  1717,  2202,  3113,
        4139,  5135,  6239,  7132,  7903,  8412,  8709,  8957,  9191,
        9343,  9479,  9600,  9641,  9680,  9719,  9768,  9834,  9913,
       10016, 10069, 10115, 10156, 10223, 10303, 11096, 11171, 11255,
       11330, 11405, 11468, 11537, 11657, 11720, 11805, 11868, 11937,
       12012, 12088, 12175, 12259, 12333, 12514, 12601, 12678, 12755,
       12827, 12898, 12953, 13012, 13059, 13095, 13123, 13170, 13204,
       13244, 13298, 13584, 13826], dtype=int32), address=array([    0,     1,     2, ..., 13824, 13825, 13826], dtype=int32))

## Check

In [10]:
rst_voxels.nvoxels == rst_voxels_h.nvoxels

True

In [11]:
import numpy as np
np.testing.assert_array_equal(rst_voxels.voxels, rst_voxels_h.voxels)

In [12]:
np.testing.assert_array_equal(rst_voxels.slice_ids, rst_voxels_h.slice_ids)

In [13]:
np.testing.assert_array_equal(rst_voxels.address, rst_voxels_h.address)

In [14]:
np.testing.assert_array_equal(rst_voxels.slice_start, rst_voxels_h.slice_start)

# Anode response

In [15]:
nslices = voxels_data.xmin.shape[0]
anode_response = rst_serial.create_anode_response(nslices, nsipms, slices)

In [16]:
anode_response[1023]

2.6653137

In [17]:
slices

ResetSlices(start=array([   0,   26,   92,  195,  326,  456,  591,  755,  955, 1190, 1433,
       1652, 1818, 1949, 2053, 2149, 2229, 2282, 2315, 2325, 2338, 2347,
       2359, 2370, 2382, 2403, 2426, 2450, 2467, 2478, 2486, 2498, 2523,
       2548, 2572, 2596, 2619, 2638, 2654, 2675, 2698, 2724, 2745, 2762,
       2778, 2801, 2824, 2850, 2872, 2894, 2910, 2933, 2950, 2973, 2992,
       3011, 3025, 3038, 3045, 3051, 3056, 3068, 3074, 3086, 3094, 3103,
       3108], dtype=int32), sensors=array([1023, 1601, 1609, ..., 1329, 1345,  379], dtype=int32), charges=array([2.6653137, 3.7328777, 5.282883 , ..., 3.0275135, 6.91506  ,
       5.1015234], dtype=float32))

## CUDA version

In [18]:
anode_d = rstf.create_anode_response(rst.cudaf, slices_data_d,
                                         rst.nsipms, nslices,
                                         np.int32(slices.charges.shape[0]))


In [19]:
total_sensors = nsipms * nslices
anode_h = cuda.from_device(anode_d, (total_sensors,), np.dtype('f4'))

In [20]:
anode_h.shape

(118272,)

## Check

In [21]:
np.testing.assert_array_almost_equal(anode_response[1792:(2*1792)], anode_h[1792:(2*1792)])

# Create cathode response

In [ ]:
energies

# Compute active sensors

In [ ]:
import invisible_cities.reco.dst_functions as dstf

pmt_param_file   = "/home/jmbenlloch/reset_data/mapas/PMT_Map_corr_keV.h5"
sipm_param_file  = "/home/jmbenlloch/reset_data/mapas/SiPM_Map_corr_z0.0_keV.h5"

sipm_param = dstf.load_xy_corrections(sipm_param_file, group="ResetMap", node="SiPM")
#dstf.load_xy_corrections(pmtcorr+".h5", group="ResetMap", node="PMT")

In [ ]:
sipms_per_voxel = int(math.floor(2 * sipm_dist / pitch) + 1)**2
voxels_per_sipm = int((2 * sipm_dist)**2 / (x_size * y_size))
xs = data_sipm.X
ys = data_sipm.Y

In [ ]:
probs, sensor_ids, voxel_starts, sensor_starts, nprobs, fwd_num = compute_probabilities(voxels[:1000], 1000, xs, ys, nsipms, sipms_per_voxel, sipm_dist, sipm_param, anode_response)

In [ ]:
sensor_ids

# Sensor probs

In [ ]:
sensor_probs, sensor_starts, sensor_starts_c, sensor_starts_c_ids = compute_sensor_probs(probs, nprobs, nsipms, voxel_starts, sensor_ids, slice_ids)

# Forward denom

In [ ]:
forward_denoms(nsipms, nslices, voxels, sensor_probs, voxel_ids, sensor_starts_c, sensor_starts_c_ids)

# MLEM step